In [1]:
# Install these if not there
# !pip install pydub
# !pip install light-the-torch
# !pip install  git+https://github.com/hmmlearn/hmmlearn.git
# !pip install  git+https://github.com/pyannote/pyannote-audio.git@develop

In [2]:
import os
# Enter either 0 or 1 below to select the GPU to use
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-07-02 10:15:43.703412: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 10:15:43.703466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 10:15:43.704997: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 10:15:43.716856: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 10:15:44.530593: W tensorflow/comp

In [3]:
from pyannote.audio import Pipeline
import whisper
import torch
from datetime import timedelta
from pydub import AudioSegment
from pathlib import Path
import re
import json
import speechbrain as sb

ImportError: /opt/jupyterhub/lib/python3.11/site-packages/torch/lib/libtorch_cuda.so: undefined symbol: ncclCommRegister

In [ ]:
device = torch.device("cuda")

In [ ]:
device

### With first usage of whisper first needs to install/update "numba.jit" - happens automatically - 1.42gb

# Generate a token from hugginface and insert it below
1. visit hf.co/pyannote/speaker-diarization and accept user conditions
2. visit hf.co/pyannote/segmentation and accept user conditions
3. visit hf.co/settings/tokens to create an access token
4. instantiate pretrained speaker diarization pipeline

# Things to change

In [ ]:
hf_token = "<HF_TOKEN>" # You need to create this and enter here as explained above
NUM_SPEAKERS = 2 # Amount of people who are speaking.
LANGUAGE = "en" # English: 'en' German: 'de'
data = "./dataFolder" # Folder with all your m4a audio files nright next to this file
dataOut = "./outFolder" # Create this folder for all the output to be saved in
types = ('**/*.m4a', '**/*.mp3', '**/*.ogg', '**/*.aac') # wav does not work right now due to how I implemented things

# convert m4a to wav and create subfolder structure and add spacer before file

In [ ]:
# function which does all steps for one file 
spacermilli = 2000
def convertAudiofiles(audio_file):
    f = f"{str(audio_file).split('.')[0]}"
    print(f.split("/")[1])
    f =  f.replace('data', 'out')
    # Create folders
    !mkdir {f} 
    ftmp = f"{f}/tmp"
    !mkdir {ftmp}
    fa = f"{f}/{str(audio_file).split('/')[1].split('.')[0]}.wav"
    print(f, fa)
    tmp = {"name": f.split("/")[1], "og_audio": fa, "tmp_folder": ftmp}
    !ffmpeg -y -i {audio_file} {fa}
    
    # tmp["proc_audio"] = tmp["og_audio"].split(".")[0] +  "_prep.wav"
    # spacer = AudioSegment.silent(duration=spacermilli)
    # audio = AudioSegment.from_wav(tmp["og_audio"]) 
    # audio.export(tmp["og_audio"], format='wav')
    return tmp

In [ ]:
audio_files = []
for fileending in types:
    pathlist = Path(data).glob(fileending)
    for audio_file in pathlist:
        audio_files.append(convertAudiofiles(audio_file))

# Append time before actual audio, else pyannot might not pick up the first 30 seconds

In [ ]:
for file in audio_files:
    processed_audio_file = file["og_audio"]
    # processed_audio_file2 = file["og_audio"].split(".")[0] +  "_prep.wav"
    # file["proc_audio"] = processed_audio_file2
    spacer = AudioSegment.silent(duration=spacermilli)
    audio = AudioSegment.from_wav(processed_audio_file) 
    audio = spacer.append(audio, crossfade=0)
    audio.export(file["og_audio"], format='wav')
audio_files

# Init Pyannot pipeline for speaker identification and whisper for transcription

In [ ]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=hf_token)
pipeline.to(torch.device("cuda"))
device = torch.device("cuda")
model = whisper.load_model('large', device = device)

# Create diarization file (when does a new speaker speak) - this will take a long while

In [ ]:
def create_diarization_file(audiofile, numspeaker):
    if numspeaker == None:
        dz = pipeline(audiofile)  
    else:
        dz = pipeline(audiofile, num_speakers=numspeaker) 
    diarization_file = audiofile.split(".")[0] + "_dia.txt"
    print(diarization_file)
    if os.path.isfile(diarization_file):
        return diarization_file
    with open(diarization_file, "w") as text_file:
        text_file.write(str(dz))
    return diarization_file

def millisec(timeStr):
    spl = timeStr.split(":")
    s = (int)((int(spl[0]) * 60 * 60 + int(spl[1]) * 60 + float(spl[2]) )* 1000)
    return s

In [ ]:
for file in audio_files:
    file["dia_file"] = create_diarization_file(file["og_audio"], NUM_SPEAKERS)
audio_files

# Create audio segments based on the previous timestamps

In [ ]:
print("test")

In [ ]:
def create_segments(file):
    dzs = open(file["dia_file"]).read().splitlines()
    groups = []
    g = []
    lastend = 0
    for d in dzs:   
        if g and (g[0].split()[-1] != d.split()[-1]):      #same speaker
            groups.append(g)
            g = []

        g.append(d)

        end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=d)[1]
        end = millisec(end)
        if (lastend > end):       #segment engulfed by a previous segment
            groups.append(g)
            g = [] 
        else:
            lastend = end
    if g:
        groups.append(g)
    audio = AudioSegment.from_wav(file["og_audio"])
    
    gidx = -1
    tmp_files = []
    for g in groups:
        start = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
        end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[-1])[1]
        start = millisec(start) #- spacermilli
        end = millisec(end)  #- spacermilli
        gidx += 1
        fname = f"{file['tmp_folder']}/{gidx}.wav"
        tmp_files.append(fname)
        audio[start:end].export(fname, format='wav')
    print(tmp_files[:10])
    return tmp_files, groups 

In [ ]:
for file in audio_files:
    file["tmp_wavs"], file["groups"] = create_segments(file)
audio_files

# Create Transcripts for each segment

In [ ]:
def create_segment_transcripts(file, language='en'):
    # print(len(file["tmp_wavs"]))
    for i in range(len(file["tmp_wavs"])):
        audiof = str(i) + '.wav'
        segment_audio = f"{file['tmp_folder']}/{audiof}"
        result = model.transcribe(audio=segment_audio, language=language, word_timestamps=True)#, initial_prompt=result.get('text', ""))
        o = f"{file['tmp_folder']}/"
        tmp_files = []
        # print(f"{o}{i}.json")
        with open(f"{o}{i}.json", "w") as outfile:
            tmp_files.append(str(outfile))
            json.dump(result, outfile, indent=4) 
    return tmp_files

In [ ]:
len(audio_files)

In [ ]:
audio_files

In [ ]:
for file in audio_files:
    file["tmp_jsons"] = create_segment_transcripts(file, language=LANGUAGE)

# Create txt file with all spoken text split by speaker

You can edit names of the speakers below

In [ ]:
speakers = {
    'SPEAKER_00': ('SPEAKER_00', '#e1ffc7', 'darkgreen'),
    'SPEAKER_01': ('SPEAKER_01', 'white', 'darkorange'),
    'SPEAKER_02': ('SPEAKER_02', 'lightblue', 'darkblue'),
    'SPEAKER_03': ('SPEAKER_03', 'lightpink', 'darkred'),
    'SPEAKER_04': ('SPEAKER_04', '#f0e68c', 'goldenrod'),
    'SPEAKER_05': ('SPEAKER_05', 'lightcoral', 'darkred'),
    'SPEAKER_06': ('SPEAKER_06', '#98fb98', 'forestgreen'),
    'SPEAKER_07': ('SPEAKER_07', '#dda0dd', 'purple'),
    'SPEAKER_08': ('SPEAKER_08', '#ffcccb', 'darkred'),
    'SPEAKER_09': ('SPEAKER_09', 'lightcyan', 'darkcyan')
}
def_boxclr = 'white'
def_spkrclr = 'orange'

In [ ]:
transcriptFolder = f"{dataOut}/transcripts"

In [ ]:
!mkdir {transcriptFolder}

In [ ]:
def timeStr(t):
    return '{0:02d}:{1:02d}:{2:06.2f}'.format(round(t // 3600), round(t % 3600 // 60), t % 60)
def create_capfile(file):
    txt = list("")
    gidx = -1
    last_speaker = ""
    for g in file["groups"]:  
        shift = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
        shift = millisec(shift) - spacermilli #the start time in the original video
        shift=max(shift, 0)

        gidx += 1
        o = f"{file['tmp_folder']}/"
        captions = json.load(open(f"{o}{gidx}.json"))['segments']
        
        if captions:
            speaker = g[0].split()[-1]
            boxclr = def_boxclr
            spkrclr = def_spkrclr
        else:
            continue
        if speaker in speakers:
            speaker, boxclr, spkrclr = speakers[speaker] 

        tmp_txt = ""
        tmp_timestamp = ""
        
        for c in captions:
            start = shift + c['start'] * 1000.0 
            start = start / 1000.0   #time resolution ot youtube is Second.            
            end = (shift + c['end'] * 1000.0) / 1000.0   
            if speaker != last_speaker:
                # print(speaker)
                tmp_txt = ""
                tmp_timestamp = f'[{timeStr(start)} --> {timeStr(end)}] [{speaker}]'
            tmp_txt += f'{c["text"]}'

            last_speaker = speaker
        txt.append(f'{tmp_timestamp}{tmp_txt}\n')


    capfile = f"{transcriptFolder}/{file['name']}_capspeaker.txt"

    with open(capfile, "w", encoding='utf-8') as file:
        s = "".join(txt)
        file.write(s)
        print(f'captions saved to {capfile}:')

# Create the actual transcribed files with speaker differentiation

In [ ]:
for file in audio_files:
    create_capfile(file)

In [ ]:
print("done")